Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 06/2019.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

## Phần 3: Hệ phương trình Euler hai chiều, vncfd_2D
`Nếu không đào bới thì còn gì là thú vị!`

Bạn hãy tưởng tượng việc `nghiên cứu CFD` giống như hành trình `đi tìm kho báu`. Đứng trong một khu rừng nhiệt đới rậm rạp, trên bãi biển của một hòn đảo hoang sơ hay đứng trước đại dương rộng lớn, ta tự hỏi không biết nên làm những gì, đi đến đâu... Chúng ta cần những lời chỉ dẫn, cần `một chiếc bản đồ kho báu`. Những bài báo, những quyển sách về lưu chất, phương pháp tính, về CFD chính là tờ bản đồ kho báu đó. Chúng ta đọc, nghiên cứu và đi theo, chúng ta đã tới được `những nơi được đánh dấu` trên bản đồ. Thế nhưng, trong tay ta `không có một tấc sắt`, không có một `chiếc xẻng` để đào bới... `Chương trình CFD` chính là chiếc xẻng để ta khám phá kho báu, đào hay chôn. Mà `nếu không đào bới thì còn gì là thú vị!`. Chúng ta cần rèn một cái xẻng. Nó có thể được rèn bằng Python, bằng C++, bằng FORTRAN, MATLAB... hay một `vật liệu` nào khác. Khi chúng ta đã biết cách rèn xẻng, biết cách sử dụng một dạng vật liệu thì việc chuyển sang rèn bằng một vật liệu khác không quá phức tạp. 

Tóm lại là ta cần một **công cụ** để đào sâu nghiên cứu CFD.

Khóa học **Thực hành CFD với Python!** sẽ cho bạn những lời chỉ dẫn, một mảnh của tấm bản đồ và cách rèn xẻng cho hành trình `đi tìm kho báu`.
 
Chúng ta chỉ mới đi được 2 bước: ở **[phần I](https://github.com/SangVn/CFD_Notebook_P1) và [phần II](https://github.com/SangVn/CFD_Notebook_P2)** ta đã tìm hiểu về phương pháp tính, ứng dụng giải các phương trình, hệ phương trình đặc trưng từ đơn giản tới phức tạp, ta đã dừng lại ở hệ phương trình Euler một chiều. Còn đó hệ phương trình Euler hai chiều, ba chiều; hệ phương trình Navier-Stokes; các mô hình rối Spalart-Allmaras, k-omega, k-epsilon, SST, LES, DNS....

Thực ra 3D so với 2D thì `chỉ là` thêm 1D, Navier-Stokes hay các mô hình rối so với Euler thì chỉ là thêm một biến, hai biến. Tất nhiên trong cái từ `chỉ là` ấy còn nhiều khác biệt, nhưng khi ta đã có nền tảng để phát triển ta có thể làm được nhiều thứ.

Phần 3 `Thực hành CFD với Python!` sẽ xoay quanh việc xây dựng chương trình giải hệ phương trình Euler 2D. Việc chuyển từ 1D sang 2D là một bước đi quan trọng và thú vị. Phần này tập trung giải thích cấu trúc chương trình, cấu trúc dữ liệu, nhiều phần lý thuyết chỉ dừng lại ở việc giới thiệu tài liệu để bạn đọc tự tham khảo. Chương trình sử dụng **phương pháp thể tích hữu hạn; phương pháp tính dòng Godunov, Roe; sơ đồ hiện, bậc một theo thời gian; tái cấu trúc nghiệm bậc một Godunov, lưới có cấu trúc** và được viết bằng ngôn ngữ `python2.7`. Ta cũng sẽ làm quen với phần mềm `ParaView` để biểu diễn và xử lý kết quả. Các bài toán ví dụ bao gồm: **dòng chảy trên âm qua dốc; dòng chảy bao hình trụ, NACA profile và tàu vũ trụ Crew Dragon**.

<img src='img/Bai_18_0.png'>

Để chuẩn bị cho phần 3, các bạn hãy cài đặt và học cách sử dụng **PyCharm** để viết code và **ParaView** để biểu diễn và xử lý dữ liệu CFD.

**Tài liệu tham khảo:**
- Eleuterio F. Toro `Riemann Solvers and Numerical Methods for Fluid Dynamics`
- Randall J. Leveque `Finite-Volume Methods for Hyperbolic Problems`
- H. K. Versteeg, W. Malalasekera `An introduction to Computational Fluid Dynamics. The Finite Volume Method`
- Katake Masatsuka `I do like CFD. Governing Equations and Exact Solutions`
- F. Moukalled, L. Mangani, M. Darwish `The Finite Volume Method in Computational Fluid Dynamics. An Advanced Introduction with OpenFOAM and Matlab`


# Bài 18. Hệ phương trình Euler hai chiều, phương pháp giải

Các kiến thức cơ bản đã được giới thiệu trong phần II, sau đây ta tóm tắt một số điểm cơ bản.

# 1. Hệ phương trình Euler hai chiều (2D)

Đã được giới thiệu trong [bài 16, phần II](https://nbviewer.jupyter.org/github/SangVn/CFD_Notebook_P2/blob/master/Bai_16.ipynb)

Dòng chảy của khí lý tưởng trong hai chiều không gian được mô tả bởi hệ phương trình Euler 2D:
$$\frac{\partial U}{\partial t} + \frac{\partial F}{\partial x} + \frac{\partial G}{\partial y} = 0\qquad(1)$$
với 
\begin{align}
U & = \begin{pmatrix} \rho \\ \rho u \\ \rho v \\ \rho e\end{pmatrix}, &
F & = \begin{pmatrix} \rho u \\ \rho u^2 + p \\ \rho u v \\ \rho uh\end{pmatrix} &
G & = \begin{pmatrix} \rho v \\ \rho uv \\ \rho v^2 + p \\ \rho vh\end{pmatrix} &
\end{align}
và $h = e + \frac{p}{\rho}$, $e = \varepsilon + \frac{u^2+v^2}{2}$, $\varepsilon = \frac{p}{(\gamma-1)\rho}$;

trong đó: $\rho$ - khối lượng riêng; $u, v$ - vận tốc theo phương x, y; $p$ - áp suất; $e$ - năng lượng một đơn vị khối lượng chất khí; $\varepsilon$ - nội năng; $h$ - enthalpy.

# 2. Phương pháp thể tích hữu hạn

Đã được giới thiệu trong [bài 11, phần II](https://nbviewer.jupyter.org/github/SangVn/CFD_Notebook_P2/blob/master/Bai_11.ipynb)

<img src='img\Bai_18_1.png' width=600>

Xét `thể tích hữu hạn` ABCD (hình phải) không nằm trên biên, có các `bề mặt` AB, BC, CD, DA; lấy tích phân phương trình (1):

$$\int_{ABCD} \left(\frac{\partial U}{\partial t} + \frac{\partial F}{\partial x} + \frac{\partial G}{\partial y} \right)dxdy = 0$$

áp dụng định lý Green ta thu được phương trình dạng tích phân:
$$\frac{d}{dt}\int U dV + \oint_{ABCD} \vec F. \vec n ds = 0 \qquad (2)$$
với $\vec F = (F, G)$, $\vec n$ - vector pháp tuyến đơn vị, $\vec n ds = (dy, -dx)$.

Phương trình (2) mô tả định luật `bảo toàn` khối lượng, động lượng và năng lượng. Nó gồm hai thành phần tương ứng sự biến đổi theo thời gian và dòng đi qua các bề mặt.

### Sơ đồ sai phân 
Sơ đồ sai phân của phương trình (2) với xấp xỉ thời gian bậc một có dạng:
$$\frac{U^{n+1} - U^n}{\Delta t} + \frac{1}{V_{ABCD}} (\vec F.\vec S_{AB} + \vec F.\vec S_{BC} + \vec F.\vec S_{CD} + \vec F.\vec S_{DA}) = 0 \qquad (3)$$
với:

$U^n = \frac{1}{V_{ABCD}}\int U dV$ - giá trị trung bình của U trong thể tích đang xét tại thời điểm $t$, $V_{ABCD}$ - thể tích hữu hạn (trường hợp 2D - diện tích ABCD);

$\vec S = \vec n . S$ - vector pháp tuyến bề mặt S, có độ lớn bằng diện tích bề mặt (2D - độ dài AB, BC, CD, DA).

Ta có công thức xác định $U^{n+1}$ tại thời điểm $t+\Delta t$:

$$U^{n+1} = U^n + \frac{\Delta t}{V_{ABCD}}\sum {\vec F.\vec S_n} \qquad (4)$$
với $\vec S_n = -\vec n . S = -(nx, ny).S$

### Công thức tính dòng qua bề mặt
Công thức tính dòng qua một đơn vị diện tích bề mặt:
\begin{align}
Flux = \vec F.(\vec n) = (F, G).(nx,ny) & = \begin{pmatrix} \rho(un_x+vn_y) \\ \rho u(un_x+vn_y) + pn_x \\ \rho v(un_x+vn_y) + pn_y \\ \rho h(un_x+vn_y)\end{pmatrix}, &
\end{align}

Trong đó $un_x+vn_y = V_n$ - vận tốc của dòng chảy theo phương vuông góc với bề mặt. Ta có:

\begin{align}
Flux & = \begin{pmatrix} \rho V_n \\ \rho uV_n + pn_x \\ \rho vV_n + pn_y \\ \rho hV_n\end{pmatrix}, &
\end{align}


## Các điểm chú ý

So với phương pháp giải hệ phương trình Euler 1D ở phần 2, khi giải hệ trong không gian hai chiều (2D) chúng ta sẽ cần phải quan tâm kỹ tới các vấn đề: đặc tính hình học; lưới; cấu trúc dữ liệu; điều kiện biên; biểu diễn, xử lý kết quả. 

Ta sẽ bắt đầu thực hành với việc tạo lưới cho bài toán dòng chảy trên âm ở bài sau.


# [Bài 19. Dựng lưới có cấu trúc ](Bai_19.ipynb)


